# LAB 1b:  Prepare babyweight dataset.

**Learning Objectives**

1. Setup up the environment
1. Preprocess natality dataset
1. Augment natality dataset
1. Create the train and eval tables in BigQuery
1. Export data from BigQuery to GCS in CSV format


## Introduction 
In this notebook, we will prepare the babyweight dataset for model development and training to predict the weight of a baby before it is born.  We will use BigQuery to perform data augmentation and preprocessing which will be used for AutoML Tables, BigQuery ML, and Keras models trained on Cloud AI Platform.

In this lab, we will set up the environment, create the project dataset, preprocess and augment natality dataset, create the train and eval tables in BigQuery, and export data from BigQuery to GCS in CSV format.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/1b_prepare_data_babyweight.ipynb).

## Set up environment variables and load necessary libraries

Check that the Google BigQuery library is installed and if not, install it. 

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [3]:
!pip install --user google-cloud-bigquery==1.25.0


     |████████████████████████████████| 169 kB 7.1 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.38.0 requires google-resumable-media<2.0dev,>=1.2.0, but you have google-resumable-media 0.5.1 which is incompatible.


**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Import necessary libraries.

In [4]:
import os
from google.cloud import bigquery

## Lab Task #1: Set environment variables.

Set environment variables so that we can use them throughout the entire lab. We will be using our project name for our bucket, so you only need to change your project and region.

In [5]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-00-0db9b1bc58c6


In [6]:
# TODO: Change environment variables
PROJECT = "qwiklabs-gcp-00-0db9b1bc58c6"  # REPLACE WITH YOUR PROJECT NAME
BUCKET = PROJECT  # REPLACE WITH YOUR BUCKET NAME, DEFAULT BUCKET WILL BE PROJECT ID
REGION = "us-central1"  # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["BUCKET"] = PROJECT if BUCKET == "BUCKET" else BUCKET # DEFAULT BUCKET WILL BE PROJECT ID
os.environ["REGION"] = REGION

if PROJECT == "cloud-training-demos":
    print("Don't forget to update your PROJECT name! Currently:", PROJECT)

## The source dataset

Our dataset is hosted in [BigQuery](https://cloud.google.com/bigquery/). The CDC's Natality data has details on US births from 1969 to 2008 and is a publically available dataset, meaning anyone with a GCP account has access. Click [here](https://console.cloud.google.com/bigquery?project=bigquery-public-data&p=publicdata&d=samples&t=natality&page=table) to access the dataset.

The natality dataset is relatively large at almost 138 million rows and 31 columns, but simple to understand. `weight_pounds` is the target, the continuous value we’ll train a model to predict.

## Create a BigQuery Dataset and Google Cloud Storage Bucket 

A BigQuery dataset is a container for tables, views, and models built with BigQuery ML. Let's create one called __babyweight__ if we have not already done so in an earlier lab. We'll do the same for a GCS bucket for our project too.

In [8]:
%%bash

## Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w babyweight)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: babyweight"
    
    bq --location=US mk --dataset \
        --description "Babyweight" \
        $PROJECT:babyweight
    echo "Here are your current datasets:"
    bq ls
fi
    
## Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
    
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

Creating BigQuery dataset titled: babyweight
Dataset 'qwiklabs-gcp-00-0db9b1bc58c6:babyweight' successfully created.
Here are your current datasets:
  datasetId   
 ------------ 
  babyweight  
  taxifare    
Bucket exists, let's not recreate it.


## Create the training and evaluation data tables

Since there is already a publicly available dataset, we can simply create the training and evaluation data tables using this raw input data. First we are going to create a subset of the data limiting our columns to `weight_pounds`, `is_male`, `mother_age`, `plurality`, and `gestation_weeks` as well as some simple filtering and a column to hash on for repeatable splitting.

* Note:  The dataset in the create table code below is the one created previously, e.g. "babyweight".

### Lab Task #2: Preprocess and filter dataset

We have some preprocessing and filtering we would like to do to get our data in the right format for training.

Preprocessing:
* Cast `is_male` from `BOOL` to `STRING`
* Cast `plurality` from `INTEGER` to `STRING` where `[1, 2, 3, 4, 5]` becomes `["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]`
* Add `hashcolumn` hashing on `year` and `month`

Filtering:
* Only want data for years later than `2000`
* Only want baby weights greater than `0`
* Only want mothers whose age is greater than `0`
* Only want plurality to be greater than `0`
* Only want the number of weeks of gestation to be greater than `0`

In [14]:
%%bigquery
SELECT
    *
FROM
#     publicdata.samples.natality
    publicdata.samples.INFORMATION_SCHEMA.COLUMNS
WHERE 
    table_name = 'natality'
ORDER BY column_name


Downloading: 100%|██████████| 31/31 [00:01<00:00, 20.73rows/s]


,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,publicdata,samples,natality,alcohol_use,22,YES,BOOL,NEVER,None,None,NO,None,NO,NO,NaN
1,publicdata,samples,natality,apgar_1min,11,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
2,publicdata,samples,natality,apgar_5min,12,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
3,publicdata,samples,natality,born_alive_alive,25,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
4,publicdata,samples,natality,born_alive_dead,26,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
5,publicdata,samples,natality,born_dead,27,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
6,publicdata,samples,natality,child_race,8,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
7,publicdata,samples,natality,cigarette_use,20,YES,BOOL,NEVER,None,None,NO,None,NO,NO,NaN
8,publicdata,samples,natality,cigarettes_per_day,21,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
9,publicdata,samples,natality,day,4,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN


In [23]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data AS
SELECT
    weight_pounds, 
    # Cast is_male from BOOL to STRING
    CASE WHEN is_male THEN 'men' ELSE 'wemen' END AS is_male,
    mother_age, 
    # Cast plurality from INTEGER to STRING where [1, 2, 3, 4, 5] becomes 
    # ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]
    CASE
        WHEN plurality = 1 THEN 'Single(1)'
        WHEN plurality = 2 THEN 'Twins(2)'
        WHEN plurality = 3 THEN 'Triplets(3)'
        WHEN plurality = 4 THEN 'Quadruplets(4)'
        WHEN plurality = 5 THEN 'Quintuplets(5)'
    END AS plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(
        CAST(year AS STRING),
        CAST(month AS STRING)))
    ) AS hashmonth # Add hashcolumn hashing on year and month
FROM
    publicdata.samples.natality
WHERE 1=1
    AND year > 2000 # Only want data for years later than 2000
    AND weight_pounds > 0 # Only want baby weights greater than 0
    AND mother_age > 0 # Only want mothers whose age is greater than 0
    AND plurality > 0 # Only want plurality to be greater than 0
    AND gestation_weeks > 0 # Only want the number of weeks of gestation to be greater than 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1937.92query/s]                        


""


### Lab Task #3: Augment dataset to simulate missing data

Now we want to augment our dataset with our simulated babyweight data by setting all gender information to `Unknown` and setting plurality of all non-single births to `Multiple(2+)`.

In [33]:
%%bigquery
SELECT
    weight_pounds, 
    # Cast is_male from BOOL to STRING
    'Unknown' AS is_male,
    mother_age, 
    # plurality of all non-single births to `Multiple(2+)`
    CASE
        WHEN plurality != 'Single(1)' THEN 'Multiple(2+)'
    END AS plurality,
    gestation_weeks
FROM
    babyweight.babyweight_data

Downloading: 100%|██████████| 33011359/33011359 [00:13<00:00, 2391718.39rows/s]


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,2.839554,Unknown,25,Multiple(2+),27
1,3.110723,Unknown,25,Multiple(2+),31
2,1.000899,Unknown,25,None,19
3,6.937947,Unknown,25,None,46
4,2.480200,Unknown,25,None,30
...,...,...,...,...,...
33011354,7.813183,Unknown,24,None,44
33011355,8.483388,Unknown,24,None,44
33011356,6.000983,Unknown,24,None,44
33011357,7.936641,Unknown,24,None,44


In [36]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_augmented_data AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data
UNION ALL
SELECT
    weight_pounds, 
    # Cast is_male from BOOL to STRING
    'Unknown' AS is_male,
    mother_age, 
    # plurality of all non-single births to `Multiple(2+)`
    CASE
        WHEN plurality != 'Single(1)' THEN 'Multiple(2+)'
    END AS plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1538.63query/s]                        


""


In [41]:
%%bigquery
SELECT COUNT(*) FROM babyweight.babyweight_augmented_data

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.12s/rows]


,f0_
0,66022718


In [43]:
%%bigquery
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    MOD(hashmonth, 4) AS mod_hashmonth
FROM
    babyweight.babyweight_augmented_data
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  5.87rows/s]


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mod_hashmonth
0,6.874013,men,14,Single(1),29,2
1,2.160530,men,15,Single(1),26,0
2,7.561856,men,43,Single(1),29,1
3,5.500533,men,15,Single(1),25,3
4,2.936557,men,42,Single(1),29,1
5,6.624891,men,15,Single(1),31,3
6,1.999593,men,42,Single(1),25,1
7,6.966607,men,15,Single(1),45,0
8,3.249614,men,13,Single(1),26,2
9,5.500533,men,46,Single(1),31,2


### Lab Task #4: Split augmented dataset into train and eval sets

Using `hashmonth`, apply a modulo to get approximately a 75/25 train/eval split.

### Split augmented dataset into train dataset

#### **Exercise**:   **RUN** the query to create the training data table.

In [46]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_train AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    MOD(hashmonth, 4) < 3

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1832.37query/s]                        


""


In [47]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT count(*) FROM babyweight.babyweight_data_train
# LIMIT 0

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.35s/rows]


,f0_
0,52885324


### Split augmented dataset into eval dataset

#### **Exercise**:   **RUN** the query to create the evaluation data table.

In [48]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_eval AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    MOD(hashmonth, 4) = 3

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1722.74query/s]                        


""


## Verify table creation

Verify that you created the dataset and training data table.


In [49]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT count(*) FROM babyweight.babyweight_data_train
# LIMIT 0

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.39s/rows]


,f0_
0,52885324


In [50]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT count(*) FROM babyweight.babyweight_data_eval


Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.50s/rows]


,f0_
0,13137394


## Lab Task #5: Export from BigQuery to CSVs in GCS

Use BigQuery Python API to export our train and eval tables to Google Cloud Storage in the CSV format to be used later for TensorFlow/Keras training. We'll want to use the dataset we've been using above as well as repeat the process for both training and evaluation data.

In [52]:
# Construct a BigQuery client object.
client = bigquery.Client()

dataset_name = 'babyweight'

# Create dataset reference object
dataset_ref = client.dataset(
    dataset_id=dataset_name, project=client.project)

extract_jobs = []

# Export both train and eval tables
for step in ['train', 'eval']:
    destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", "{}*.csv".format(step))
    table_name = "babyweight_data_{}".format(step)
    table_ref = dataset_ref.table(table_name)
    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
    )  # API request
    extract_jobs.append(extract_job)
    
print('waiting for jobs')
for extract_job in extract_jobs:
    extract_job.result()  # Waits for job to complete.

    print("Exported {}:{}.{} to {}".format(
        client.project, dataset_name, table_name, destination_uri))

waiting for jobs
Exported qwiklabs-gcp-00-0db9b1bc58c6:babyweight.babyweight_data_eval to gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/eval*.csv
Exported qwiklabs-gcp-00-0db9b1bc58c6:babyweight.babyweight_data_eval to gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/eval*.csv


## Verify CSV creation

Verify that we correctly created the CSV files in our bucket.

In [53]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*.csv

gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/eval000000000000.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/eval000000000001.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000000.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000001.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000002.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000003.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000004.csv
gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/data/train000000000005.csv


In [54]:
%%bash
gsutil cat gs://${BUCKET}/babyweight/data/train000000000000.csv | head -5

weight_pounds,is_male,mother_age,plurality,gestation_weeks
4.1005980732,wemen,44,Triplets(3),33
2.06352677232,men,45,Triplets(3),30
3.62219496466,men,42,Triplets(3),31
5.3792791928,wemen,50,Triplets(3),35


In [55]:
%%bash
gsutil cat gs://${BUCKET}/babyweight/data/eval000000000000.csv | head -5

weight_pounds,is_male,mother_age,plurality,gestation_weeks
4.0565056208,men,46,Triplets(3),32
4.74875712348,men,46,Triplets(3),34
1.81219979364,men,44,Triplets(3),32
2.19800875214,wemen,45,Triplets(3),31


## Lab Summary: 
In this lab, we setup our environment, created a BigQuery dataset, preprocessed and augmented the natality dataset, created train and eval tables in BigQuery, and exported data from BigQuery to GCS in CSV format.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License